In [1]:
import pandas as pd
import numpy as np
import copy

from games4e import Game, GameState


In [ ]:
class Mancala(Game):
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones
        self.p1_pits_index = [0, pits_per_player-1]
        self.p1_mancala_index = pits_per_player
        self.p2_pits_index = [pits_per_player+1, len(board)-2] 
        self.p2_mancala_index = len(board)-1
        
        # Zero out the Mancala pits
        board[self.p1_mancala_index] = 0
        board[self.p2_mancala_index] = 0
        
        # Create initial GameState
        moves = list(range(self.p1_pits_index[0], self.p1_pits_index[1]+1))
        self.initial = GameState(to_move=1, utility=0, board=board, moves=moves)

    def actions(self, state: GameState) -> list[int]:
        # GameState = namedtuple('GameState', 'to_move, utility, board, moves')
        # available actions based on the player to move and the board state
        if state.to_move == 1:
            return [pit for pit in range(self.p1_pits_index[0], self.p1_pits_index[1]+1) if state.board[pit] > 0]
        else:
            return [pit for pit in range(self.p2_pits_index[0], self.p2_pits_index[1]+1) if state.board[pit] > 0]
    
    def result(self, state: GameState, move: int) -> GameState:
        # update the board state after a move is made
        new_state = copy.deepcopy(state)
        board = new_state.board
        current_player = state.to_move
        
        # Convert move to board index if player 2
        pit = move if current_player == 1 else move
        
        # Get stones from pit
        stones = board[pit]
        board[pit] = 0
        
        # Distribute stones
        current_pit = pit
        while stones > 0:
            current_pit = (current_pit + 1) % len(board)
            # Skip opponent's mancala
            if current_player == 1 and current_pit == self.p2_mancala_index:
                continue
            if current_player == 2 and current_pit == self.p1_mancala_index:
                continue
            board[current_pit] += 1
            stones -= 1
            
        # Handle capture
        # Check if the last stone lands in an empty pit
        if board[current_pit] == 1:
            # Check if the last stone lands in the player's own pits
            if current_player == 1 and current_pit in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
                # Get the opposite pit index
                opposite_pit = self.p2_pits_index[1] - current_pit
                # Check if the opposite pit is not empty
                if board[opposite_pit] != 0:
                    # Capture the stones in the opposite pit and the last stone
                    board[self.p1_mancala_index] += board[opposite_pit] + 1
                    board[opposite_pit] = 0
                    board[current_pit] = 0
                    
            # Check if the last stone lands in the player's own pits
            if current_player == 2 and current_pit in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
                # Get the opposite pit index
                opposite_pit = self.p1_pits_index[1] - (current_pit - 7)
                # Check if the opposite pit is not empty
                if board[opposite_pit] != 0:
                    # Capture the stones in the opposite pit and the last stone
                    board[self.p2_mancala_index] += board[opposite_pit] + 1
                    board[opposite_pit] = 0
                    board[current_pit] = 0
        # Switch player
        next_player = 1 if current_player == 2 else 2
        
        # Get available moves for next player
        if next_player == 1:
            moves = [pit for pit in range(self.p1_pits_index[0], self.p1_pits_index[1]+1) if board[pit] > 0]
        else:
            moves = [pit for pit in range(self.p2_pits_index[0], self.p2_pits_index[1]+1) if board[pit] > 0]
            
        return GameState(to_move=next_player, utility=0, board=board, moves=moves)
        
    def utility(self, state: GameState, player: int) -> int:
        # return the utility of the game state for the given player
        utility = state.board[self.p1_mancala_index] - state.board[self.p2_mancala_index]
        return utility if player == 1 else -utility
    
    def terminal_test(self, state: GameState) -> bool:
        # check if the game is in a terminal state
        for pit in range(self.p1_pits_index[0], self.p1_pits_index[1]+1):
            if state.board[pit] != 0:
                return False
        for pit in range(self.p2_pits_index[0], self.p2_pits_index[1]+1):
            if state.board[pit] != 0:
                return False
        return True
    
    def to_move(self, state: GameState) -> int:
        # return the player whose turn it is
        return state.to_move    
    
    def display(self, state: GameState):
        # print the game state
        player_1_pits = state.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = state.board[self.p1_mancala_index]
        player_2_pits = state.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = state.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if state.to_move == 1 else 'P2'
        print('Turn: ' + turn)
    